In [2]:
import sys
sys.path.insert(0, '/home/ah1114/LanguageOfLife/BioDL')
#and import all the stuff
from data import *
from fastai.callbacks import *
from datetime import datetime 

In [3]:
path = Path('./') 
#model_path = Path('/home/ah1114/LanguageOfLife/FrameClassification/models/frameclas_k1')
pretrained_path = Path('/home/ah1114/LanguageOfLife/FrameClassification/models/keep_frameclas_k1_constvalidset')
#log_path = Path('/home/ah1114/LanguageOfLife/FrameClassification/train_logs/frameclas_k1')
vocab_path = Path('/home/ah1114/LanguageOfLife/vocabs')
vocab_name = vocab_path/'ngs_vocab_k1_withspecial.npy'

#data_path = Path('/scratch/ah1114/LoL/data/GTDBrepCDS_chunked_csv/')
data_path = Path('/scratch/ah1114/LoL/data/testCDS_csv/')

#lr_plot_out = '/home/ah1114/LanguageOfLife/FrameClassification/lrplot_frameclas_k1.png'
#if stuck with 128GB, might try doing it in shuffled batches of files?
bs=2048 #2048 worked in previous tests, 4096 had memory error
ksize=1
stride=1
max_seqs=100 
lrate=1e-2 #judging from ANI 0.95 no bidir cutoff; this should maybe be higher
drop_mult=0.3 #haven't tuned this at all yet
num_cycles=1

tok = BioTokenizer(ksize=ksize, stride=stride)
if vocab_name.is_file():
    voc = np.load(vocab_name)
    model_voc = BioVocab(voc)
else:
    model_voc = BioVocab.create_from_ksize(ksize=ksize)
    np.save(vocab_name, model_voc.itos)

In [4]:
data = BioClasDataBunch.from_multiple_csv(path=data_path, 
                                        text_cols='seq', label_cols='frame',
                                        valid_pct=0.2, 
                                        tokenizer=tok, vocab=model_voc,
                                        max_seqs_per_file=max_seqs, bs=bs
                                            )

In [5]:
data

TextClasDataBunch;

Train: LabelList (237 items)
x: BioTextList
xxbos G A A A G T G G A G C G A T T C A A A C G A T T C G T C A A G G G T T T A T T G A T A T C T C C C C C G A C C G G C G G G T G C A G G C C A T T A T T A T T T G C T G G T T G T T T G G A T C C T T T A T T G A A G G C G C T T C C G G C T G G G G G A C A C C G G C T G C C G T A C T A G C A C C T T T G C,xxbos T A A T G C A A G G A A T A A C C C A A T G A T C T G C T T T T T A G T G A T C A C T T C T T T T A G A A A G A G A C C A G A T A A C A A A A C A A T T A A C A T C G G A T T T G C C G C T A T A A A C A A A G A G C T T T T A A T C A C T G G C G C A T G C T T G C T G G C C A A G A A G A A A C A A A T,xxbos A G C C T C T A G C G A A G C A A G G T G T C G T A G G A G A G T G G A T T G C A A C C T C T A G G A T T G A G A T T G A A C A G G C G A G G C T T C T T A C G T T A A A A G C T G C T T C G A T G A T G G A T A C A G T T G G,xxbos A A G C G A T C T A A C T C T T C T C C T T C T A C T C G C T C G A T C T C T T T T G 

In [15]:
config = awd_lstm_clas_config.copy() #make sure this is awd_lstm_clas_config for classification!
config['bidir'] = False #this can be True for classification, but if you're using a pretrained LM you need to keep if = False because otherwise your matrix sizes don't match
learn = text_classifier_learner(data, AWD_LSTM, drop_mult=drop_mult, model_dir=".", config=config, pretrained=False)
learn.load(pretrained_path)

RNNLearner(data=TextClasDataBunch;

Train: LabelList (237 items)
x: BioTextList
xxbos G A A A G T G G A G C G A T T C A A A C G A T T C G T C A A G G G T T T A T T G A T A T C T C C C C C G A C C G G C G G G T G C A G G C C A T T A T T A T T T G C T G G T T G T T T G G A T C C T T T A T T G A A G G C G C T T C C G G C T G G G G G A C A C C G G C T G C C G T A C T A G C A C C T T T G C,xxbos T A A T G C A A G G A A T A A C C C A A T G A T C T G C T T T T T A G T G A T C A C T T C T T T T A G A A A G A G A C C A G A T A A C A A A A C A A T T A A C A T C G G A T T T G C C G C T A T A A A C A A A G A G C T T T T A A T C A C T G G C G C A T G C T T G C T G G C C A A G A A G A A A C A A A T,xxbos A G C C T C T A G C G A A G C A A G G T G T C G T A G G A G A G T G G A T T G C A A C C T C T A G G A T T G A G A T T G A A C A G G C G A G G C T T C T T A C G T T A A A A G C T G C T T C G A T G A T G G A T A C A G T T G G,xxbos A A G C G A T C T A A C T C T T C T C C T T C T A C T C G C T C G A T 

In [16]:
interp = TextClassificationInterpretation.from_learner(learn)

In [17]:
df = pd.read_csv('/scratch/ah1114/LoL/data/testCDS_csv/GCA_000006945.2_ASM694v2.csv',nrows=100)
df.head()

,genome,seq,header,frame
0,GCA_000006945.2_ASM694v2,GTAATGCCAAGAGTCCGCCAGAGGATACGCTCAATGCCAGCCACGC...,lcl|AE006468.2_cds_AAL19650.1_686[subset=8..15...,-2
1,GCA_000006945.2_ASM694v2,TTAACCGCGCCAATCGCGCCCCGTTCAGCGCCTTTCTTCGTTTACA...,lcl|AE006468.2_cds_AAL20739.1_1775[subset=700....,3
2,GCA_000006945.2_ASM694v2,TGATGCAGTTGACTAAAAAAGTTGGCAAGGTTATCAGCGGGATAGC...,lcl|AE006468.2_cds_AAL21667.1_2703[subset=452....,-2
3,GCA_000006945.2_ASM694v2,CGAAAGAATGTCGATTCGTTCCGTCGGCGTGCGGCAGGATACCGGG...,lcl|AE006468.2_cds_AAL19763.1_799[subset=1760....,2
4,GCA_000006945.2_ASM694v2,TGGCGGGCAAAGGCGAGATAGCCTCCTGCGCGAAACCCACGGCGAT...,lcl|AE006468.2_cds_AAL21020.1_2056[subset=1753...,3


In [18]:
df['seq'][0]

'GTAATGCCAAGAGTCCGCCAGAGGATACGCTCAATGCCAGCCACGCCCGGCAACGGAACGGCGGCGATGAGCCTTGCCAGGCCCGAACCCAATGGTTTCGCCAGTACCAATAAAATCAGTAAAAAACTGGCGATCAGTAAAAATCCTTGC'

In [21]:
for x in range(0,15):
    print('frame is',df['frame'][x])
    
    interp.show_intrinsic_attention(df['seq'][x])

frame is -2


frame is 3


frame is -2


frame is 2


frame is 3


...

frame is -3


frame is -1


frame is -1


frame is -1


frame is -1


In [ ]:
interp.show_intrinsic_attention(df['seq'][1])